#MIT 6.036 Spring 2019: Lab 13#

This colab notebook provides an alternative space to complete [Lab 13](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week13/week13_lab/).

## <section>**Setup**</section>

Run the next code block to download and import the code for this lab.

In [1]:
!rm -rf code_for_lab13* __MACOSX data .DS_Store
!wget --quiet https://introml_oll.odl.mit.edu/6.036/static/labs/lab13/code_for_lab13.zip
!unzip code_for_lab13.zip
!mv code_for_lab13/* .

from non_parametric import auto_data_and_labels, auto_data_all, one_hot, standard, raw, auto_xval

from IPython.display import clear_output
# clear_output()

Archive:  code_for_lab13.zip
   creating: code_for_lab13/
  inflating: code_for_lab13/auto-mpg.tsv  
  inflating: code_for_lab13/non_parametric.py  
avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot
Loaded non_parametric.py


# 1) Basic operation

Look at the code directory and open the file `non_parametric.py`. You will see an implementation of the recursive (greedy) decision-tree building algorithm described in lecture, and an implementation of a nearest neighbor classifier.

We will be running this algorithm on the Auto-MPG dataset that we used in Lab 3 and HW 3; you should look back at the description on the Lab 3 and your results from HW 3 re-familiarize yourself with the data. In the `non_parametric.py` file, towards the end is the code from HW 3 that we used to read the data into Numpy matrices. The one difference is that we are reading the individual points into the *rows* of the matrices rather than the columns as we did before.

Find the place where `features` are defined. This is as before, you can choose there whether a feature is kept in its `raw` form, converted to `standard` or to `one_hot`. Note that the call


```
auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)
```

will create the data and labels matrices. Crucially, it will also print which feature index corresponds to which original feature. For example:


```
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot
```


The first number is the feature index, which plays a prominent role in the decision-tree construction. Remember that the one_hot transformation produces a number of features with values either 0 or 1 corresponding to a discrete value of the original feature, so:



```
2 cylinders 5.0 one_hot
```

means that feature index 2 corresponds to 1 when cylinders is equal to 5.0 and 0 otherwise. In the default setting all the features are raw. Note that by default we're ignoring model name and model year, as we discussed in Lab 3.

Run the following code block to try out the above.

In [2]:
features = [('cylinders', one_hot),
            ('displacement', standard),
            ('horsepower', standard),
            ('weight', standard),
            ('acceleration', standard),
            ## Drop model_year by default
            ## ('model_year', raw),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot


Two useful functions defined in the file are:



*   `auto_test(pct=0.25)` splits the auto data (randomly) into training and test sets of fractions `1-pct` and `pct`, respectively, and runs the decision tree construction algorithm on the training data and evaluates the resulting tree on the test set. By default, it prints the tree to depth 5 (see the definition of `dt_eval`) and prints (and returns) the accuracy
*   `auto_xval(k=10)` does k-fold cross validation. It also calls `dt_eval` and print the trees and accuracy. At the end the function returns the average accuracy across the runs.

A tree (that corresponds to the feature definitions above) might look like this (note that `n` is the number of remaining points in the subtree):



```
Feat 7 < 0.0476422698221 [ n= 294.0 ]
   Feat 7 < -0.251184192133 [ n= 171.0 ]
     Feat 2 < 0.5 [ n= 142.0 ]
       Feat 8 < -0.940473332238 [ n= 8.0 ]
         => 1.00 [ n= 4.0 ]
         => 0.00 [ n= 4.0 ]
       Feat 7 < -0.887737602216 [ n= 134.0 ]
         => 1.00 [ n= 74.0 ]
         Feat 7 < -0.442150215158 [ n= 60.0 ]
           Feat 8 < 0.837889708284 [ n= 42.0 ]
             Depth > 5
             Depth > 5
           => 1.00 [ n= 18.0 ]
     Feat 8 < 0.239053174231 [ n= 29.0 ]
       Feat 5 < -0.628541123663 [ n= 17.0 ]
         Feat 5 < -0.853672534932 [ n= 11.0 ]
           => 0.00 [ n= 4.0 ]
           Feat 6 < 1.35064454164 [ n= 7.0 ]
             Depth > 5
             Depth > 5
         => 0.00 [ n= 6.0 ]
       Feat 8 < 0.583837845352 [ n= 12.0 ]
         => 1.00 [ n= 4.0 ]
         Feat 6 < 0.961017160039 [ n= 8.0 ]
           => 1.00 [ n= 2.0 ]
           Feat 6 < 1.14084518232 [ n= 6.0 ]
             Depth > 5
             Depth > 5
   Feat 8 < 1.54560561217 [ n= 123.0 ]
     Feat 8 < 0.638277530266 [ n= 117.0 ]
       => 0.00 [ n= 102.0 ]
       Feat 5 < -0.203108971485 [ n= 15.0 ]
         => 0.00 [ n= 14.0 ]
         => 1.00 [ n= 1.0 ]
     Feat 8 < 2.36220088587 [ n= 6.0 ]
       Feat 3 < 0.5 [ n= 4.0 ]
         => 0.00 [ n= 3.0 ]
         => 1.00 [ n= 1.0 ]
       => 1.00 [ n= 2.0 ]
```


The printing is done by recursively exploring the tree, printing the test at a node (`Feat i < th [ n= n ]`, then going to the left child (the Yes node) first and then the right child (the No node). When it gets to a leaf in the tree, it prints '=> p [ n= n ]', where *p* is the prediction (probability of positive label).

Make sure that you can read the tree. Note: Why are some of the thresholds negative values?

###1A) What do the classes that we are trying to predict in this problem mean? (Review the description in [Lab 3](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week3/week3_lab/))

###1B) Run a 10-fold cross validation with the one-hot and standardized features (run the next code block to do this). This will print 10 trees, each with an accuracy, then an overall accuracy.



*   Look back to [HW 3](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week3/week3_homework/) and find the best accuracy we got with Average Perceptron for those features and compare.
*   Look at the tests being used at the top layers of the trees, what features are the tests splitting on?
*   Look at the leaves of the trees; what does the float between 0 and 1 represent? How do you decide what class to output?



In [3]:
features = [('cylinders', one_hot),
            ('displacement', standard),
            ('horsepower', standard),
            ('weight', standard),
            ('acceleration', standard),
            ## Drop model_year by default
            ## ('model_year', raw),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10)

avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot
 Feat 1 < 0.5 [ n= 352.0 ]
   Feat 5 < -0.6285411236631759 [ n= 176.0 ]
     Feat 8 < -0.8497405240477289 [ n= 21.0 ]
       => 1.00 [ n= 5.0 ]
       Feat 8 < 0.9467690781120286 [ n= 16.0 ]
         Feat 7 < -0.1285887205618948 [ n= 13.0 ]
           => 0.00 [ n= 5.0 ]
           Feat 7 < -0.024264689465024896 [ n= 8.0 ]
             Depth > 5
             Depth > 5
         => 1.00 [ n= 3.0 ]
     Feat 8 < 0.51125159

0.9080769230769231

###1C) Use the following small two-feature set in the next code block and repeat the process in 1B. Draw the top two layers of the tree, what do they represent?

In [4]:
# A small feature set
features = [('weight', standard),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10)

avg and std {'weight': (2977.5841836734694, 848.3184465698365)}
entries in one_hot field {'origin': [1.0, 2.0, 3.0]}
0 weight std
1 origin 1.0 one_hot
2 origin 2.0 one_hot
3 origin 3.0 one_hot
 Feat 0 < -0.25118419213335774 [ n= 352.0 ]
   Feat 0 < -0.8877376022159538 [ n= 170.0 ]
     Feat 0 < -1.0067966659536245 [ n= 87.0 ]
       => 1.00 [ n= 58.0 ]
       Feat 0 < -1.0056178633423605 [ n= 29.0 ]
         => 0.00 [ n= 1.0 ]
         => 1.00 [ n= 28.0 ]
     Feat 0 < -0.8847905956877937 [ n= 83.0 ]
       => 0.00 [ n= 1.0 ]
       Feat 0 < -0.8240822612076942 [ n= 82.0 ]
         => 1.00 [ n= 11.0 ]
         Feat 0 < -0.38144188067803714 [ n= 71.0 ]
           Feat 0 < -0.8181882481513739 [ n= 55.0 ]
             Depth > 5
             Depth > 5
           Feat 3 < 0.5 [ n= 16.0 ]
             Depth > 5
             Depth > 5
   Feat 0 < 0.32937609391419514 [ n= 182.0 ]
     Feat 0 < 0.04764226982208318 [ n= 57.0 ]
       Feat 0 < -0.1562905819266004 [ n= 33.0 ]
         Feat 0 < -0.

0.8417948717948718

###1D) Does it change anything if we use `raw` for origin and/or standardize weight (run the next code block to do this)? Explain.

In [5]:
# The choice of feature processing for each feature, mpg is always raw and
# does not need to be specified.  Other choices are standard and one_hot.
# 'name' is not numeric and would need a different encoding.
features = [('weight', raw),
            ('origin', raw)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10)

avg and std {}
entries in one_hot field {}
0 weight raw
1 origin raw
 Feat 0 < 2764.5 [ n= 352.0 ]
   Feat 0 < 2224.5 [ n= 170.0 ]
     Feat 0 < 2123.5 [ n= 87.0 ]
       => 1.00 [ n= 58.0 ]
       Feat 0 < 2124.5 [ n= 29.0 ]
         => 0.00 [ n= 1.0 ]
         => 1.00 [ n= 28.0 ]
     Feat 0 < 2227.0 [ n= 83.0 ]
       => 0.00 [ n= 1.0 ]
       Feat 0 < 2278.5 [ n= 82.0 ]
         => 1.00 [ n= 11.0 ]
         Feat 0 < 2654.0 [ n= 71.0 ]
           Feat 0 < 2283.5 [ n= 55.0 ]
             Depth > 5
             Depth > 5
           Feat 1 < 2.5 [ n= 16.0 ]
             Depth > 5
             Depth > 5
   Feat 0 < 3257.0 [ n= 182.0 ]
     Feat 0 < 3018.0 [ n= 57.0 ]
       Feat 0 < 2845.0 [ n= 33.0 ]
         Feat 0 < 2803.5 [ n= 9.0 ]
           => 0.25 [ n= 4.0 ]
           => 0.00 [ n= 5.0 ]
         Feat 0 < 2996.5 [ n= 24.0 ]
           Feat 0 < 2959.5 [ n= 21.0 ]
             Depth > 5
             Depth > 5
           => 1.00 [ n= 3.0 ]
       Feat 1 < 1.5 [ n= 24.0 ]
         =

0.8443589743589743

###1E) When might we want to use a decision tree classifier as opposed to an averaged perceptron or neural net classifier?

# 2) Parameter Tuning

Note the three variables defined as class variables at the top of the definition of the class `DTN` in the code file.



*   `N_THRESHOLD = 4 # don't split if node has fewer examples than this`
*  `H_THRESHOLD = .01 # don't split if node has entropy less than this`
*  `H_REDUCTION_THRESHOLD = .001 # don't split if it doesn't reduce entropy by this`

## Code for DTN

In [6]:
# Binary (+1, -1) prediction.  This assumes that the data points are
# the ROWS of the matrix!!  The last column of the data matrix are the
# labels.

class DTN:
    N_THRESHOLD = 4 # don't split if node has fewer examples than this
    H_THRESHOLD = .01 # don't split if node has entropy less than this
    H_REDUCTION_THRESHOLD = .001 # don't split if it doesn't reduce H by this
    index = 0
    def __init__(self, data=None, config = None):
        self.config = config
        if config != None:
            self.N_THRESHOLD = config[0]
            self.H_THRESHOLD = config[1]
            self.H_REDUCTION_THRESHOLD = config[2]
        DTN.index += 1
        self.index = DTN.index          # unique number
        self.data = data                # store the data
        self.p = None                   # prob of positive - look at set_h
        if data is not None:
            self.n = float(data.shape[0]) # number of data points
            self.indices = range(data.shape[1]-1) # feature indices
            self.set_h()                           # compute entropy
        self.splits = {}
        # The test is data[:,fi] < th
        self.fi = None                  # feature index
        self.th = None                  # threshold
        self.lc = None                  # left child
        self.rc = None                  # right child
        self.parent = None              # parent

    # Create split on feat i at value th
    def split(self, i, th):
        self.fi = i
        self.th = th
        self.lc = DTN(d[d[:, i] < th], config = self.config)
        self.rc = DTN(d[d[:, i] >= th], config = self.config)
        self.splits[i].remove(th)

    # Evalute candidate split by weighted average entropy
    def split_eval(self, i, th):
        lc = DTN(self.data[self.data[:, i] < th], config = self.config)
        rc = DTN(self.data[self.data[:, i] >= th], config = self.config)
        pl = lc.n / self.n
        pr = 1.0 - pl
        avgH = pl * lc.h + pr * rc.h
        return avgH, lc, rc

    # Entropy of class labels in this node, assumes 1, -1
    def set_h(self):
        b = .001
        npos = np.sum(self.data[:,-1] == 1) # count labels = 1
        p = (npos + b) / (self.n + b + b)
        self.p = p
        self.h = - (p * np.log(p) + (1 - p) * np.log(1 - p))

    def build_tree(self):
        if self.h < self.H_THRESHOLD or self.n <= self.N_THRESHOLD:
            return
        # Find the best split
        (i, th, (h, lc, rc)) = argmax([(i, th, self.split_eval(i, th)) \
                                                   for i in self.indices \
                                                   for th in self.get_splits(i)],
                                      lambda x: -x[2][0]) # x=(a,b,(h,c,d))

        if (self.h - h) < self.H_REDUCTION_THRESHOLD:
            return
        # Recurse again!
        self.fi = i
        self.th = th
        self.lc = lc
        self.rc = rc
        self.lc.parent = self
        self.rc.parent = self
        self.lc.build_tree()
        self.rc.build_tree()

    # The candidate splits between the values in the training set.
    def get_splits(self, i):
        if i not in self.splits:
            d = np.sort(self.data[:,i], axis = None)
            d1 = d[:-1]
            d2 = d[1:]
            self.splits[i] = (d1 + d2) / 2.0
        return self.splits[i]

    # Classify a data point
    def classify(self, x):
        if self.fi == None:             # leaf
            return self.p               # prob of posistive
        elif x[self.fi] < self.th:
            return self.lc.classify(x)  # satisfies test, left branch
        else:
            return self.rc.classify(x)  # fails test, right branch

    def display(self, depth=0, max_depth=3):
        if depth > max_depth:
            print(depth*'  ', 'Depth >', max_depth)
            return
        if self.fi is None:
            print(depth*'  ', '=>', "%.2f"%self.p, '[', 'n=', self.n, ']')
            return
        print(depth*'  ', 'Feat', self.fi, '<', self.th, '[', 'n=', self.n, ']')
        self.lc.display(depth+1, max_depth)
        self.rc.display(depth+1, max_depth)

## Questions for problem 2

###2A) By changing these parameters, theoretically what is the minimum training error you could get? Is this a good thing?

###2B) From a learning algorithm perspective, what is the purpose of these values?

###2C) Try changing these values, for example `N_THRESHOLD` over [1, 2, 4], `H_THRESHOLD` over [.01, .1, .25] and `H_REDUCTION_THRESHOLD` over [.001, .01, .1]. Note the effect on the accuracy of 10-fold cross-validation and on the result trees. To expedite the process, you can use the `config` input variable to `auto_xval` to change the values, so for example you can set `config = [4, .01, .001]` to run `auto_xval` using the default values. Be prepared to explain your findings. How big are the trees on average with the lowest values of the parameters vs the highest values of the parameters?

You can set `verbose='first_tree'` to output only the first tree.

Run the following code blocks with different values for `config`.

In [7]:
features = [('cylinders', one_hot),
            ('displacement', standard),
            ('horsepower', standard),
            ('weight', standard),
            ('acceleration', standard),
            ## Drop model_year by default
            ## ('model_year', raw),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot


In [8]:
auto_xval(auto_data, auto_labels, k=10, verbose='first_tree', config=[4, .01, .001])

 Feat 1 < 0.5 [ n= 352.0 ]
   Feat 5 < -0.6285411236631759 [ n= 176.0 ]
     Feat 8 < -0.8497405240477289 [ n= 21.0 ]
       => 1.00 [ n= 5.0 ]
       Feat 8 < 0.9467690781120286 [ n= 16.0 ]
         Feat 7 < -0.1285887205618948 [ n= 13.0 ]
           => 0.00 [ n= 5.0 ]
           Feat 7 < -0.024264689465024896 [ n= 8.0 ]
             Depth > 5
             Depth > 5
         => 1.00 [ n= 3.0 ]
     Feat 8 < 0.5112515988005719 [ n= 155.0 ]
       Feat 7 < -0.5588516736732791 [ n= 125.0 ]
         => 0.33 [ n= 3.0 ]
         => 0.00 [ n= 122.0 ]
       Feat 5 < -0.4282403827544763 [ n= 30.0 ]
         Feat 7 < 0.06886071682483638 [ n= 25.0 ]
           => 0.50 [ n= 2.0 ]
           => 0.00 [ n= 23.0 ]
         Feat 7 < 1.0785051533725096 [ n= 5.0 ]
           => 1.00 [ n= 3.0 ]
           => 0.00 [ n= 2.0 ]
   Feat 7 < -0.8877376022159538 [ n= 176.0 ]
     => 1.00 [ n= 86.0 ]
     Feat 10 < 0.5 [ n= 90.0 ]
       Feat 7 < -0.5394014305874221 [ n= 66.0 ]
         Feat 8 < 1.0193553246639

0.9080769230769231

###2D) Could you change these parameters such that no splits occur? If so, how?

# 3) Nearest Neighbors
We can set the `decision_tree` parameter in `auto_xval` to `False` in order to
run a nearest neighbor model instead.

###3A) Run 10-fold cross validation using nearest neighbors and report the accuracy.

In [9]:
features = [('cylinders', one_hot),
            ('displacement', standard),
            ('horsepower', standard),
            ('weight', standard),
            ('acceleration', standard),
            ## Drop model_year by default
            ## ('model_year', raw),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)


auto_xval(auto_data, auto_labels, k=10, decision_tree=False)

avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot
Test_accuracy 0.95
Test_accuracy 0.85
Test_accuracy 0.9743589743589743
Test_accuracy 0.9743589743589743
Test_accuracy 0.8205128205128205
Test_accuracy 0.8717948717948718
Test_accuracy 0.8205128205128205
Test_accuracy 0.8974358974358975
Test_accuracy 0.9230769230769231
Test_accuracy 0.7948717948717948
Xval accuracy 0.8876923076923078


0.8876923076923078

###3B) Repeat using the small two-feature sets from 1C and 1D. How does using the unprocessed `raw` features from 1D compare to the processed features from 1C?

In [10]:
features = [('weight', standard),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10, decision_tree=False)

avg and std {'weight': (2977.5841836734694, 848.3184465698365)}
entries in one_hot field {'origin': [1.0, 2.0, 3.0]}
0 weight std
1 origin 1.0 one_hot
2 origin 2.0 one_hot
3 origin 3.0 one_hot
Test_accuracy 0.825
Test_accuracy 0.925
Test_accuracy 0.8717948717948718
Test_accuracy 0.8974358974358975
Test_accuracy 0.7948717948717948
Test_accuracy 0.8974358974358975
Test_accuracy 0.9230769230769231
Test_accuracy 0.8717948717948718
Test_accuracy 0.8974358974358975
Test_accuracy 0.7948717948717948
Xval accuracy 0.8698717948717951


0.8698717948717951

In [11]:
features = [('weight', raw),
            ('origin', raw)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10, decision_tree=False)

avg and std {}
entries in one_hot field {}
0 weight raw
1 origin raw
Test_accuracy 0.825
Test_accuracy 0.9
Test_accuracy 0.7948717948717948
Test_accuracy 0.7948717948717948
Test_accuracy 0.7692307692307693
Test_accuracy 0.9487179487179487
Test_accuracy 0.7948717948717948
Test_accuracy 0.8717948717948718
Test_accuracy 0.8461538461538461
Test_accuracy 0.7948717948717948
Xval accuracy 0.8340384615384615


0.8340384615384615

###3C) We can run k-nearest neighbors by setting the `config` parameter to desired value of k. Note that this is different from the `k` parameter, which describes the number of folds in cross-validation. Choosing odd values of k is preferred, to avoid ties in choosing the majority classification.

###Can you get a higher cross validation accuracy on the first feature set using a different value of k? If so, what k did you use and what accuracy does it achieve?

In [12]:
features = [('cylinders', one_hot),
            ('displacement', standard),
            ('horsepower', standard),
            ('weight', standard),
            ('acceleration', standard),
            ## Drop model_year by default
            ## ('model_year', raw),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)


auto_xval(auto_data, auto_labels, k=10, decision_tree=False, config=1)

avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot
Test_accuracy 0.95
Test_accuracy 0.85
Test_accuracy 0.9743589743589743
Test_accuracy 0.9743589743589743
Test_accuracy 0.8205128205128205
Test_accuracy 0.8717948717948718
Test_accuracy 0.8205128205128205
Test_accuracy 0.8974358974358975
Test_accuracy 0.9230769230769231
Test_accuracy 0.7948717948717948
Xval accuracy 0.8876923076923078


0.8876923076923078

# 4) Checkoff

Please go back to [Lab 13](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week13/week13_lab/) to ask for a checkoff!